In [6]:
import numpy as np
# FIX: Import partial_transpose from its specific location in qiskit.quantum_info.operators
from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit.quantum_info.operators import partial_transpose
from qiskit.circuit.library import GHZCircuit

# --- 1. STATE GENERATION FUNCTION ---

def ghz_mixed_state(p_noise: float, num_qubits: int = 3) -> DensityMatrix:
    """
    Generates the N-qubit GHZ state mixed with white noise (depolarizing channel).

    The state is given by: rho = (1 - p_noise)|GHZ_N><GHZ_N| + p_noise * (I / 2^N).

    Args:
        p_noise: The probability of white noise (0 <= p_noise <= 1).
        num_qubits: The number of qubits in the GHZ state (e.g., 3).

    Returns:
        DensityMatrix: The mixed quantum state.
    """
    if not 0 <= p_noise <= 1:
        raise ValueError("p_noise must be between 0 and 1.")

    # 1. Create the pure GHZ state
    ghz_circuit = GHZCircuit(num_qubits)
    ghz_vector = Statevector(ghz_circuit)
    ghz_rho = DensityMatrix(ghz_vector)

    # 2. Create the maximally mixed state (I / 2^N)
    dim = 2**num_qubits
    identity_matrix = np.eye(dim)
    maximally_mixed_rho = DensityMatrix(identity_matrix / dim)

    # 3. Apply the convex combination
    mixed_rho = (1 - p_noise) * ghz_rho + p_noise * maximally_mixed_rho

    return mixed_rho

# --- 2. PARTIAL TRANSPOSE FUNCTION ---

def partial_transpose_matrix(rho: DensityMatrix, qb_subsystem: list, total_qubits: int) -> np.ndarray:
    """
    Computes the partial transpose of a density matrix with respect to a list of qubits.

    Args:
        rho: The input density matrix (e.g., 3-qubit GHZ mixed state).
        qb_subsystem: A list of integer indices (0, 1, 2, ...) specifying the qubits
                      to transpose over (e.g., [0] for A|BC partition).
        total_qubits: The total number of qubits in the system.

    Returns:
        np.ndarray: The matrix representation of the partially transposed state.
    """
    # The partial_transpose function from qiskit.quantum_info works directly
    # on the DensityMatrix object.
    # dims parameter is automatically inferred if not provided, but we specify it
    # explicitly to avoid ambiguity with non-qubit systems.
    
    # We define the dimensions of the subsystems (all are 2-dimensional for qubits)
    dims = [2] * total_qubits 
    
    # partial_transpose requires the 'qargs' parameter, which is the subsystem to transpose
    rho_pt = partial_transpose(rho, qargs=qb_subsystem, dims=dims)
    
    return rho_pt.data

# --- 3. POSITIVE SEMIDEFINITE CHECK ---

def is_positive_semidefinite(matrix: np.ndarray, epsilon: float = -1e-9) -> bool:
    """
    Checks if a matrix is positive semidefinite by examining its eigenvalues.
    This is used to check the PPT criterion: rho_TA >= 0.

    Args:
        matrix: The input Hermitian matrix (e.g., the partially transposed state).
        epsilon: Tolerance threshold for considering small negative values as zero.
                 If min eigenvalue is >= epsilon, it's considered PSD.
                 A small negative number is used to tolerate numerical error.

    Returns:
        bool: True if all eigenvalues are non-negative (>= epsilon), False otherwise.
    """
    # Note: np.linalg.eigvalsh is used for Hermitian matrices, which is faster
    # and guarantees real eigenvalues. Density matrices and their partial transposes
    # are Hermitian.
    eigenvalues = np.linalg.eigvalsh(matrix)
    
    # Check if the smallest eigenvalue is greater than the negative tolerance
    min_eigenvalue = np.min(eigenvalues)
    
    return min_eigenvalue >= epsilon

# --- DEMONSTRATION OF THE 0.8 THRESHOLD ---

if __name__ == '__main__':
    N = 3
    print(f"--- Analyzing Noisy {N}-Qubit GHZ State (Dimension 2^{N}=8) ---")
    
    # --- Test 1: Entangled State (p_noise < 0.8) ---
    p_entangled = 0.75 # Should be entangled
    print(f"\n[Test 1] Noise Level: p_noise = {p_entangled}")
    rho_ent = ghz_mixed_state(p_entangled, N)
    
    # Choose a partition (e.g., Qubit 0 vs Qubits 1, 2)
    subsystem_to_transpose = [0] 
    rho_ent_pt = partial_transpose_matrix(rho_ent, subsystem_to_transpose, N)
    
    # Check PSD condition
    is_ppt_ent = is_positive_semidefinite(rho_ent_pt)
    min_eig_ent = np.min(np.linalg.eigvalsh(rho_ent_pt))
    
    print(f"   -> Smallest Eigenvalue of PT: {min_eig_ent:.6f}")
    print(f"   -> Is PPT (Is Separable)? {is_ppt_ent}")
    print(f"   -> Result: {'ENTANGLED (NPT)' if not is_ppt_ent else 'PPT/Separable'}")
    
    # --- Test 2: Separable State (p_noise > 0.8) ---
    p_separable = 0.85 # Should be separable
    print(f"\n[Test 2] Noise Level: p_noise = {p_separable}")
    rho_sep = ghz_mixed_state(p_separable, N)
    rho_sep_pt = partial_transpose_matrix(rho_sep, subsystem_to_transpose, N)
    
    is_ppt_sep = is_positive_semidefinite(rho_sep_pt)
    min_eig_sep = np.min(np.linalg.eigvalsh(rho_sep_pt))
    
    print(f"   -> Smallest Eigenvalue of PT: {min_eig_sep:.6f}")
    print(f"   -> Is PPT (Is Separable)? {is_ppt_sep}")
    print(f"   -> Result: {'ENTANGLED (NPT)' if not is_ppt_sep else 'PPT/Separable'}")

    # --- Test 3: Analytical Boundary (p_noise = 0.8) ---
    p_boundary = 0.80 
    print(f"\n[Test 3] Noise Level: p_noise = {p_boundary} (Analytical Boundary)")
    rho_boundary = ghz_mixed_state(p_boundary, N)
    rho_boundary_pt = partial_transpose_matrix(rho_boundary, subsystem_to_transpose, N)
    min_eig_boundary = np.min(np.linalg.eigvalsh(rho_boundary_pt))
    
    print(f"   -> Smallest Eigenvalue of PT: {min_eig_boundary:.6f}")
    print(f"   -> Interpretation: At this point, the smallest eigenvalue is approximately zero, confirming the transition point.")

ImportError: partial_transpose could not be imported from qiskit.quantum_info or qiskit.quantum_info.operators. Please upgrade qiskit or adjust imports.